# Environment: Google Colab


# Phase 1: Image Assessment and Metrics Analysis

In [1]:
import cv2
import numpy as np
import os
import glob

def image_assessment(image_path):
    """
      Analyses an image to determine the correct processing path
      Returns a dictionary containing the decision, reason, and raw metrics
    """
    img = cv2.imread(image_path)
    if img is None:
        return {"decision": "Error", "reason": "Image not found", "metrics": {}}

    # 1. Convert to grayscale for analysis
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Calculate Metrics

    # A. Brightness (average pixel intensity)
    # Range: 0 (pure black) - 255 (pure white)
    avg_brightness = np.mean(gray)

    # B. Glare (saturation check)
    # Count pixels that are 'blown out' (pure white > 250)
    # Calculate the ratio of these pixels to the total image size
    num_white_pixels = np.sum(gray > 250)
    total_pixels = gray.size
    glare_ratio = num_white_pixels / total_pixels

    # C. Blur (Laplacian variance)
    # High variance = Sharp edges, Low variance = Blurry
    # Note: Glare can artificially spike this score, which is why we check glare first.
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

    # 3. Decision Logic (Based on calibration)

    # Dark check
    # Catch pitch black/dark images
    if avg_brightness < 40:
        return {"decision": "PATH_B_LOW_LIGHT", "reason": f"Too Dark ({avg_brightness:.1f})"}

    # Glare/Overexposure check
    # Very high Laplacian variance often means harsh edges caused by glare
    # Values chosen empirically from dataset observations
    elif laplacian_var > 2400:
        return {"decision": "PATH_B_GLARE", "reason": f"Extreme Contrast/Glare ({laplacian_var:.0f})"}

    # Night glare backup
    # For night images that aren't super sharp but have bright spots like headlights
    # I restrict this to avg_brightness < 60 to avoids misclassifying daytime sky or windows as glare
    elif glare_ratio > 0.005 and avg_brightness < 60:
        return {"decision": "PATH_B_GLARE", "reason": f"Night Spots ({glare_ratio:.4f})"}

    # Blur check
    # Catch blurry/fuzzy images
    elif laplacian_var < 180:
        return {"decision": "PATH_B_DEBLUR", "reason": f"Blurry ({laplacian_var:.0f})"}

    # Standard path
    else:
        return {"decision": "PATH_A_STANDARD", "reason": "Clean Image"}

    # return {
    #     "decision": decision,
    #     "reason": reason,
    #     "metrics": {
    #         "brightness": avg_brightness,
    #         "glare": glare_ratio,
    #         "blur": laplacian_var
    #     }
    # }


In [2]:
def test_folder(folder_path):
    print(f"\n--- Testing: {os.path.basename(folder_path)} ---")
    files = glob.glob(os.path.join(folder_path, "*.*"))
    for f in files:
        if not f.lower().endswith(('.jpg', '.png', '.jpeg')):
          continue
        res = image_assessment(f)

        print(f"{os.path.basename(f):<10} -> {res['decision']:<20} | {res['reason']}")

# Hand picked subset from AOLP to calibrate the values
clear_folder_path = '/content/drive/MyDrive/FYP_Stage1_Test/Easy_Clear'
bad_folder_path = '/content/drive/MyDrive/FYP_Stage1_Test/Hard_Blurry'

## Uncomment this and run this cell for output
test_folder(clear_folder_path)
test_folder(bad_folder_path)


--- Testing: Easy_Clear ---
139.jpg    -> PATH_A_STANDARD      | Clean Image
143.jpg    -> PATH_A_STANDARD      | Clean Image
142.jpg    -> PATH_A_STANDARD      | Clean Image
138.jpg    -> PATH_A_STANDARD      | Clean Image
137.jpg    -> PATH_A_STANDARD      | Clean Image
410.jpg    -> PATH_A_STANDARD      | Clean Image
406.jpg    -> PATH_A_STANDARD      | Clean Image
407.jpg    -> PATH_A_STANDARD      | Clean Image
409.jpg    -> PATH_A_STANDARD      | Clean Image
408.jpg    -> PATH_A_STANDARD      | Clean Image
329.jpg    -> PATH_A_STANDARD      | Clean Image
330.jpg    -> PATH_A_STANDARD      | Clean Image
328.jpg    -> PATH_A_STANDARD      | Clean Image
331.jpg    -> PATH_A_STANDARD      | Clean Image
334.jpg    -> PATH_A_STANDARD      | Clean Image
332.jpg    -> PATH_A_STANDARD      | Clean Image
338.jpg    -> PATH_A_STANDARD      | Clean Image
336.jpg    -> PATH_A_STANDARD      | Clean Image
335.jpg    -> PATH_A_STANDARD      | Clean Image
337.jpg    -> PATH_A_STANDARD      | Cle

# Phase 2: Preprocessing and Enhancement

In [3]:
import cv2
import numpy as np

# Stream A and B1 (lightweight logic)

def stream_a_standard(image):
    """
      Path A: Minimal processing for already good images
    """
    # 1. Convert to Grayscale (Required for OCR later)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Light Gaussian Blur (Removes camera sensor noise)
    # Kernel size (3,3) is gentle so no losing edge definition

    # Why blurring the grayscale instead of BGR:
    # 1. Cleaner blur: Grayscale avoid colour smear so can have cleaner edges
    # 2. Faster computing: Blurring a grayscale process 1 channel only
    # 3. Consistent input: Match the processing stream for CLAHE
    processed = cv2.GaussianBlur(gray, (3, 3), 0)

    # 3. Convert back to BGR
    processed_bgr = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)

    return processed_bgr

def stream_b1_illumination(image):
    """
      - Path B1: Fix glare/low light using CLAHE for images that are too dark or have bright headlight spots
      - CLAHE operates on small tiles (grid size 8x8). It boosts details in dark areas (low light) and limits
        contrast amplification in bright spots (glare), making it effective for both conditions
    """
    # 1. Convert to Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Apply CLAHE
    # clipLimit=2.0 prevents noise amplification in flat areas.
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    # 3. Convert back to BGR for consistency
    enhanced_bgr = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

    return enhanced_bgr

In [4]:
# Clone Restormer
!git clone https://github.com/swz30/Restormer.git
%cd Restormer
!wget https://github.com/swz30/Restormer/releases/download/v1.0/motion_deblurring.pth -P Motion_Deblurring/pretrained_models

Cloning into 'Restormer'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 312 (delta 74), reused 72 (delta 72), pack-reused 197 (from 2)
Receiving objects: 100% (312/312), 1.55 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (131/131), done.
/content/Restormer
--2025-12-26 15:20:13--  https://github.com/swz30/Restormer/releases/download/v1.0/motion_deblurring.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/418793252/55c7bcd2-cb39-4d8a-adc4-acf6f6131c27?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-26T16%3A17%3A16Z&rscd=attachment%3B+filename%3Dmotion_deblurring.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-95

In [5]:
from runpy import run_path
import torch
import torch.nn.functional as F

# Stream B2 (Restormer Setup)

def load_restormer():
    """
      Loads the Restormer model into GPU memory.
    """
    # Define hyperparameters
    parameters = {
      'inp_channels':3,
      'out_channels':3,
      'dim':48,
      'num_blocks':[4,6,6,8],
      'num_refinement_blocks':4,
      'heads':[1,2,4,8],
      'ffn_expansion_factor':2.66,
      'bias':False,
      'LayerNorm_type':'WithBias',
      'dual_pixel_task':False
    }

    weights_path = 'Motion_Deblurring/pretrained_models/motion_deblurring.pth'

    # Load Architecture dynamically
    # Use run_path because Restormer is not a pip package
    restormer_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
    model = restormer_arch['Restormer'](**parameters)

    # Load state dict
    model.cuda()
    checkpoint = torch.load(weights_path)
    model.load_state_dict(checkpoint['params'])
    model.eval()

    return model

def stream_b2_deblur(image, model):
    """
      - Path B2: Removes motion blur using Restormer
      - Input: BGR image
      - Output: Deblurred BGR Image
    """
    # 1. Preprocessing: BGR > RGB > Tensor
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_ = torch.from_numpy(img).float().div(255.).permute(2,0,1).unsqueeze(0).cuda()

    # 2. Padding (Restormer requires dimensions to be multiply of 8)
    h, w = input_.shape[2], input_.shape[3]
    H, W = ((h + 8) // 8) * 8, ((w + 8) // 8) * 8
    padh = H - h if h % 8 != 0 else 0
    padw = W - w if w % 8 != 0 else 0
    input_ = F.pad(input_, (0, padw, 0, padh), 'reflect')

    # 3. Inference (disable gradient tracking in PyTorch since inference only, save gpu run time)
    with torch.no_grad():
      restored = model(input_)

    # 4. Post-processing: Unpad > Clamp > Back to BGR
    restored = torch.clamp(restored, 0, 1)
    restored = restored[:, :, :h, :w] # Crop padding
    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = cv2.cvtColor((restored[0] * 255).astype(np.uint8), cv2.COLOR_RGB2BGR)

    return restored

In [6]:
import os
import glob

# Preprocess Pipeline

def preprocess_pipeline(image_path, model, output_base_folder):
  """
    The Brain of the System:
    1. Triages the image (Phase 1).
    2. Routes it to the correct Enhancement Stream (Phase 2).
    3. Saves the result.
  """
  # 1. Run image assessment (Phase 1)
  analysis = image_assessment(image_path)
  decision = analysis["decision"]
  reason = analysis["reason"]

  img = cv2.imread(image_path)
  if img is None:
    return "Error loading image"

  # 2. Route to the correct Stream (Phase 2)

  if decision == "PATH_A_STANDARD":
    # Stream A: Lightweight
    processed_img = stream_a_standard(img)
    method_tag = "Standard"

  elif decision == "PATH_B_GLARE" or decision == "PATH_B_LOW_LIGHT":
    # Stream B1: CLAHE
    processed_img = stream_b1_illumination(img)
    method_tag = "CLAHE"

  elif decision == "PATH_B_DEBLUR":
    # Stream B2: Restormer
    processed_img = stream_b2_deblur(img, model)
    method_tag = "Restormer"

  save_folder = output_base_folder
  if not os.path.exists(save_folder):
    os.makedirs(save_folder)

  filename = os.path.basename(image_path)
  save_path = os.path.join(save_folder, filename)
  cv2.imwrite(save_path, processed_img)

  return f"[{method_tag}] {reason}"

def run_system_test(input_folder, model):
  print(f"Running on: {input_folder}")

  base_output_dir = '/content/drive/MyDrive/Phase2_Results'
  folder_name = os.path.basename(os.path.dirname(input_folder))
  dataset_output_dir = os.path.join(base_output_dir, folder_name)

  files = glob.glob(os.path.join(input_folder, "*.*"))
  count = 0

  for f in files:
    if not f.lower().endswith(('.jpg', '.png', '.jpeg')):
      continue

    log_msg = preprocess_pipeline(f, model, dataset_output_dir)

    print(f"Processed: {os.path.basename(f)} -> {log_msg}")
    count += 1

  print(f"\nDone, processed {count} images.")
  print(f"Output: {dataset_output_dir}")


if 'deblur_model' not in locals():
  deblur_model = load_restormer()

image_folders = [
  '/content/drive/MyDrive/AOLP_Dataset/Subset_AC/Image',
  '/content/drive/MyDrive/AOLP_Dataset/Subset_LE/Image',
  '/content/drive/MyDrive/AOLP_Dataset/Subset_RP/Image'
]

for folder in image_folders:
  run_system_test(folder, deblur_model)

Running on: /content/drive/MyDrive/AOLP_Dataset/Subset_AC/Image
Processed: 632.jpg -> [CLAHE] Extreme Contrast/Glare (2627)
Processed: 636.jpg -> [CLAHE] Extreme Contrast/Glare (2893)
Processed: 635.jpg -> [CLAHE] Extreme Contrast/Glare (2860)
Processed: 668.jpg -> [CLAHE] Extreme Contrast/Glare (3091)
Processed: 648.jpg -> [CLAHE] Extreme Contrast/Glare (3022)
Processed: 680.jpg -> [CLAHE] Extreme Contrast/Glare (2870)
Processed: 639.jpg -> [CLAHE] Extreme Contrast/Glare (2658)
Processed: 656.jpg -> [CLAHE] Extreme Contrast/Glare (3028)
Processed: 637.jpg -> [CLAHE] Extreme Contrast/Glare (2957)
Processed: 647.jpg -> [CLAHE] Extreme Contrast/Glare (2977)
Processed: 643.jpg -> [CLAHE] Extreme Contrast/Glare (2976)
Processed: 675.jpg -> [CLAHE] Extreme Contrast/Glare (2734)
Processed: 669.jpg -> [CLAHE] Extreme Contrast/Glare (2745)
Processed: 642.jpg -> [CLAHE] Extreme Contrast/Glare (2953)
Processed: 651.jpg -> [CLAHE] Extreme Contrast/Glare (2699)
Processed: 649.jpg -> [CLAHE] Extrem